# Install Libraries<a class="anchor" id="preliminaries"></a>
First, let's start by installing the packages that we'll need later. 

In [1]:
! pip install numpy
! pip install openai
! pip install pymongo
! pip install python-dotenv
! pip install azure-core
! pip install azure-cosmos

  Obtaining dependency information for azure-core from https://files.pythonhosted.org/packages/ef/d7/69d53f37733f8cb844862781767aef432ff3152bc9b9864dc98c7e286ce9/azure_core-1.30.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 11.1 MB/s eta 0:00:00
  Obtaining dependency information for azure-cosmos from https://files.pythonhosted.org/packages/e2/d4/38929bb3504bb9a2cb21ede7a1c652ebe7487b65da8885ea4039647bba86/azure_cosmos-4.7.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 15.9 MB/s eta 0:00:00


In [1]:
import json
import datetime
import time
import urllib 

from azure.core.exceptions import AzureError
from azure.core.credentials import AzureKeyCredential
import pymongo

from openai import AzureOpenAI
from dotenv import load_dotenv

Please use the example.env as a template to provide the necessary keys and endpoints in your own .env file.
Make sure to modify the env_name accordingly. 

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
COSMOS_MONGO_USER = os.getenv("COSMOS_MONGO_USER")
COSMOS_MONGO_PWD = os.getenv("COSMOS_MONGO_PWD")
COSMOS_MONGO_SERVER = os.getenv("COSMOS_MONGO_SERVER")

AOAI_client = AzureOpenAI(api_key=os.getenv("AZURE_OPENAI_KEY"), azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), api_version="2024-02-01",)

## Azure OpenAI <a class="anchor" id="azureopenai"></a>


In [3]:
import os
from openai import AzureOpenAI


response = AOAI_client.embeddings.create(
    input = "The name of the person is surya. he has various skills in azure, openai, machine learning.",
    model= "text-embedding-ada-002"
)

print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "embedding": [
        -0.00080497767,
        -0.00838328,
        0.011010672,
        -0.026355188,
        -0.011951929,
        0.018662615,
        -0.03125785,
        0.010651776,
        -0.013062476,
        -0.015628925,
        0.029686831,
        0.004134081,
        0.01461318,
        0.007157614,
        0.01274421,
        -0.020463869,
        0.014816329,
        0.0017758603,
        0.016874906,
        -0.024459131,
        -0.020369066,
        -0.008295248,
        -0.00678856,
        -0.0014711369,
        -0.01680719,
        0.017755218,
        0.011213821,
        -0.022305753,
        0.013069248,
        -0.022061974,
        0.015764358,
        -0.014504834,
        0.01120705,
        -0.003285934,
        -0.021073317,
        -0.0022481815,
        0.02561031,
        0.026666684,
        -0.00033836995,
        -0.006510923,
        0.027289674,
        0.010333509,
        -0.0018469624,
        -0.02798038,
        -0.0

In [4]:
response = response.model_dump_json(indent=2)

In [5]:
import json
response_json = json.loads(response)

# [{"id": 1, "title": "Azure App Service", "content": "Azure App Service is a fully managed platform for building, deploying, and scaling web apps. You can host web apps, mobile app backends, and RESTful APIs. It supports a variety of programming languages and frameworks, such as .NET, Java, Node.js, Python, and PHP. The service offers built-in auto-scaling and load balancing capabilities. It also provides integration with other Azure services, such as Azure DevOps, GitHub, and Bitbucket.", "category": "Web", "titleVector":
response_json_final = {"id": 1, "title": "Surya", "content":"The name of the person is surya. he has various skills in azure, openai, machine learning."}
response_json_final["c_vector"] = response_json["data"][0]["embedding"]

In [6]:
response_json_final

{'id': 1,
 'title': 'Surya',
 'content': 'The name of the person is surya. he has various skills in azure, openai, machine learning.',
 'c_vector': [-0.00080497767,
  -0.00838328,
  0.011010672,
  -0.026355188,
  -0.011951929,
  0.018662615,
  -0.03125785,
  0.010651776,
  -0.013062476,
  -0.015628925,
  0.029686831,
  0.004134081,
  0.01461318,
  0.007157614,
  0.01274421,
  -0.020463869,
  0.014816329,
  0.0017758603,
  0.016874906,
  -0.024459131,
  -0.020369066,
  -0.008295248,
  -0.00678856,
  -0.0014711369,
  -0.01680719,
  0.017755218,
  0.011213821,
  -0.022305753,
  0.013069248,
  -0.022061974,
  0.015764358,
  -0.014504834,
  0.01120705,
  -0.003285934,
  -0.021073317,
  -0.0022481815,
  0.02561031,
  0.026666684,
  -0.00033836995,
  -0.006510923,
  0.027289674,
  0.010333509,
  -0.0018469624,
  -0.02798038,
  -0.015412233,
  -0.0044658906,
  -0.012669722,
  -0.02536653,
  -0.009730834,
  0.022116147,
  -0.0060098227,
  0.009189104,
  -0.028359592,
  -0.012270195,
  -0.010448

# Load data and create embeddings <a class="anchor" id="loaddata"></a>
Here we load a sample dataset containing descriptions of Azure services, then we use Azure OpenAI to create vector embeddings from this data.

In [7]:
def generate_embeddings(text):
    '''
    Generate embeddings from string of text.
    This will be used to vectorize data and user input for interactions with Azure OpenAI.
    '''
    response = AOAI_client.embeddings.create(input=text, model="text-embedding-ada-002")
    embeddings =response.model_dump()
    time.sleep(0.5) 
    return embeddings['data'][0]['embedding']

In [8]:
# # Generate embeddings for title and content fields
# n = 0
# for item in data:
#     n+=1
#     title = item['title']
#     content = item['content']
#     title_embeddings = generate_embeddings(title)
#     content_embeddings = generate_embeddings(content)
#     item['titleVector'] = title_embeddings
#     item['contentVector'] = content_embeddings
#     item['@search.action'] = 'upload'
#     print("Creating embeddings for item:", n, "/" ,len(data), end='\r')
# # Save embeddings to sample_text_w_embeddings.json file
# with open("../../DataSet/AzureServices/text-sample_w_embeddings.json", "w") as f:
#     json.dump(data, f)

In [9]:
# Take a peek at one data item with embeddings created
# print(json.dumps(data[0], indent=2))

# Connect and setup Cosmos DB for MongoDB vCore

## Set up the connection

In [10]:
mongo_conn = "mongodb+srv://"+urllib.parse.quote(COSMOS_MONGO_USER)+":"+urllib.parse.quote(COSMOS_MONGO_PWD)+"@"+COSMOS_MONGO_SERVER
mongo_client = pymongo.MongoClient(mongo_conn)

/var/folders/9w/58ml_m490y304l00j5v7xblm0000gn/T/ipykernel_6119/3440047033.py:2: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  mongo_client = pymongo.MongoClient(mongo_conn)


##  Set up the DB and collection

In [11]:
# create a database called TutorialDB
db = mongo_client['ExampleDB']

# Create collection if it doesn't exist
COLLECTION_NAME = "ExampleCollection"

collection = db[COLLECTION_NAME]

if COLLECTION_NAME not in db.list_collection_names():
    # Creates a unsharded collection that uses the DBs shared throughput
    db.create_collection(COLLECTION_NAME)
    print("Created collection '{}'.\n".format(COLLECTION_NAME))
else:
    print("Using collection: '{}'.\n".format(COLLECTION_NAME))

Using collection: 'ExampleCollection'.



In [12]:
# Use only if re-reunning code and want to reset db and collection
collection.drop_indexes()
mongo_client.drop_database("ExampleDB")

## Create the vector index

In [13]:
db.command({
  'createIndexes': 'ExampleCollection',
  'indexes': [
    {
      'name': 'VectorSearchIndex',
      'key': {
        "c_vector": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-ivf',
        'numLists': 1,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
})

{'raw': {'defaultShard': {'numIndexesBefore': 1,
   'numIndexesAfter': 2,
   'createdCollectionAutomatically': True,
   'ok': 1}},
 'ok': 1}

## Upload data to the collection
A simple `insert_many()` to insert our data in JSON format into the newly created DB and collection.

In [14]:
collection.insert_many([response_json_final])

InsertManyResult([ObjectId('666a302501491be94efe3856')], acknowledged=True)

# Vector Search in Cosmos DB for MongoDB vCore

In [5]:
# Simple function to assist with vector search
def vector_search(query, num_results=5):
    query_embedding = generate_embeddings(query)
    embeddings_list = []
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": query_embedding,
                    "path": "c_vector",
                    "k": num_results#, #, "efsearch": 40 # optional for HNSW only 
                    #"filter": {"title": {"$ne": "Azure Cosmos DB"}}
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results

Let's run a test query below.

In [16]:
query = "What is the name of the person?"#"What are the services for running ML models?"
results = vector_search(query)
for result in results: 
    print(f"Similarity Score: {result['similarityScore']}")  
    print(f"Title: {result['document']['title']}")  
    print(f"Content: {result['document']['content']}")  

Similarity Score: 0.8078892574508341
Title: Surya
Content: The name of the person is surya. he has various skills in azure, openai, machine learning.


# Q&A over the data with GPT-3.5

In [1]:
import json
import datetime
import time
import urllib 

from azure.core.exceptions import AzureError
from azure.core.credentials import AzureKeyCredential
import pymongo

from openai import AzureOpenAI
from dotenv import load_dotenv

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
COSMOS_MONGO_USER = os.getenv("COSMOS_MONGO_USER")
COSMOS_MONGO_PWD = os.getenv("COSMOS_MONGO_PWD")
COSMOS_MONGO_SERVER = os.getenv("COSMOS_MONGO_SERVER")

AOAI_client = AzureOpenAI(api_key=os.getenv("AZURE_OPENAI_KEY"), azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), api_version="2024-02-01",)

In [3]:
# Simple function to assist with vector search
def vector_search(query, num_results=5):
    query_embedding = generate_embeddings(query)
    embeddings_list = []
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": query_embedding,
                    "path": "c_vector",
                    "k": num_results#, #, "efsearch": 40 # optional for HNSW only 
                    #"filter": {"title": {"$ne": "Azure Cosmos DB"}}
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results

In [4]:
def generate_embeddings(text):
    '''
    Generate embeddings from string of text.
    This will be used to vectorize data and user input for interactions with Azure OpenAI.
    '''
    response = AOAI_client.embeddings.create(input=text, model="text-embedding-ada-002")
    embeddings =response.model_dump()
    time.sleep(0.5) 
    return embeddings['data'][0]['embedding']

In [5]:
mongo_conn = "mongodb+srv://"+urllib.parse.quote(COSMOS_MONGO_USER)+":"+urllib.parse.quote(COSMOS_MONGO_PWD)+"@"+COSMOS_MONGO_SERVER
mongo_client = pymongo.MongoClient(mongo_conn)
db = mongo_client['azure']

# Create collection if it doesn't exist
COLLECTION_NAME = "azure_collection"

collection = db[COLLECTION_NAME]

/var/folders/9w/58ml_m490y304l00j5v7xblm0000gn/T/ipykernel_1451/2310975429.py:2: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  mongo_client = pymongo.MongoClient(mongo_conn)


In [8]:
#This function helps to ground the model with prompts and system instructions.

def generate_completion(vector_search_results, user_prompt):
    system_prompt = '''
    You are an intelligent assistant for Microsoft Azure services.
    You are designed to provide helpful answers to user questions about Azure services given the information about to be provided.
        - Only answer questions related to the information provided below, provide at least 3 clear suggestions in a list format.
        - Write two lines of whitespace between each answer in the list.
        - If you're unsure of an answer, you can say ""I don't know"" or ""I'm not sure"" and recommend users search themselves."
        - Only provide answers that have products that are part of Microsoft Azure and part of these following prompts.
    '''

    messages=[{"role": "system", "content": system_prompt}]
    for item in vector_search_results:
        messages.append({"role": "system", "content": item['document']['content']})
    messages.append({"role": "user", "content": user_prompt})
    response = AOAI_client.chat.completions.create(model="gpt-35-turbo", messages=messages,temperature=0)
    
    return response.dict()

In [9]:
# Create a loop of user input and model output. You can now perform Q&A over the sample data!

user_input = ""
print("*** Please ask your model questions about Azure services. Type 'end' to end the session.\n")
user_input = input("User prompt: ")
while user_input.lower() != "end":
    search_results = vector_search(user_input)
    completions_results = generate_completion(search_results, user_input)
    print("\n")
    print(completions_results['choices'][0]['message']['content'])
    user_input = input("User prompt: ")

*** Please ask your model questions about Azure services. Type 'end' to end the session.



User prompt:  what are ai services?




Azure AI services are cloud-based artificial intelligence services provided by Microsoft Azure. These services enable developers to incorporate cognitive intelligence into their applications without requiring extensive AI or data science expertise. Azure AI services offer a range of capabilities such as natural language processing, speech recognition, computer vision, and decision-making algorithms. These services provide pre-built APIs and models that can be easily integrated into applications, allowing developers to leverage AI capabilities without having to build them from scratch.


KeyboardInterrupt: Interrupted by user